In [ ]:
import openai
import os
import base64
import requests
from PIL import Image
from io import BytesIO

API_KEY = ""
# openai.api_key = API_KEY
# client = openai.OpenAI(api_key=API_KEY)

# response = client.images.generate(
#     model="dall-e-3",
#     prompt="Singapore airlines zepellins flying over a bizzare city with buildings made of ice cream. The image should fully occupy the entire canvas without any frames, borders, or cropped areas. No blank spaces or artificial framing. Focus on simplicity, bold shapes, and strong contrast to enchance clarity and visual appeal. Avoid excessive detail or complex gradients, ensuring the design works well with flat, vibrant colors.",
#     # size="1024x1024"  # options: "256x256", "512x512", "1024x1024"
#     size="1792x1024"
# )

# # Get image URL
# image_url = response.data[0].url
# response = requests.get(image_url, verify=False)
# img = Image.open(BytesIO(response.content))

In [19]:
from datetime import datetime, timedelta

def get_weekdays_in_month():
    current_day = datetime.now()
    # Get all weekdays in current month
    weekdays = []
    month_start = current_day.replace(day=1)
    month_end = (month_start.replace(day=28) + timedelta(days=4)).replace(day=1) - timedelta(days=1)

    for day in range(1, month_end.day + 1):
        date = current_day.replace(day=day)
        # 0-4 represents Monday-Friday
        if date.weekday() < 5:
            weekdays.append(day)
    weekdays_morning = [f"{day}_morning.png" for day in weekdays]
    weekdays_afternoon = [f"{day}_afternoon.png" for day in weekdays]
    return weekdays_morning + weekdays_afternoon

# weekdays = get_weekdays_in_month()
# weekdays



In [ ]:
from plugins.ai_image.ai_image import AIImage
import asyncio
import os

current_day = datetime.now()
yymm = current_day.strftime("%Y_%m")

OUTPUT_FOLDER = f"/Users/teyang_lau/Pictures/yymm"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

async def generate_image(model="dall-e-3", quality="hd", orientation="horizontal"):
    try:
        client = openai.OpenAI(api_key=API_KEY)
        text_prompt = AIImage.fetch_image_prompt(client, "")
        image = AIImage.fetch_image(
            client,
            text_prompt,
            model="dall-e-3",
            quality="hd",
            orientation="horizontal",
        )
    except Exception as e:
        raise RuntimeError(f"Open AI request failure, {str(e)}")
    return image, text_prompt

async def main():
    tasks = []
    filenames = get_weekdays_in_month()
    for _ in range(filenames):
        tasks.append(generate_image())
    results = await asyncio.gather(*tasks)
    for result in results:
        image, text_prompt = result
        image.save(f"{OUTPUT_FOLDER}/{text_prompt}.png")
    return results

# Run the async function
results = await main()
for i, result in enumerate(results, 1):
    print(f"Text prompt {i}: {result[1]}")


RuntimeError: Open AI request failure, please check logs.